# **SAUDI ARABIA RBC estimation**

##### PSME 2023-2024 Macroeconomics TD-Group: 
- Cameron Ann Jessie
- Landini Elia
- Molitor Maurice

##### The following notebook houses our Saudi Arabia RBC Estimation Project, an endeavor aimed at employing Real Business Cycle (RBC) modeling to analyze economic fluctuations in Saudi Arabia over a timeframe spanning over 40 years. Through statistical and econometric techniques, we aim to uncover the underlying factors driving macroeconomic dynamics in the Saudi context.

## **INTRODUCTORY SETUP**

In [37]:
# login and link to our shared repository available on GitHub
!git init
!git config --global user.email "e.lando2001@gmail.com"
!git config --global user.name "EliaLand"
!git add SaudiArabia_RBC_estimation.ipynb  
!git commit -m "Initial commit"
!git remote add origin https://github.com/EliaLand/Uni-projects.git
!git push -u origin master 

Reinitialized existing Git repository in C:/Users/eland/Uni-projects/.git/


[main 37f1465] Initial commit
 1 file changed, 312 insertions(+), 111 deletions(-)


error: remote origin already exists.
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/EliaLand/Uni-projects.git'


In [38]:
# Packages download
!pip install pandas
!pip install pandas_datareader
!pip install prettytable
!pip install statsmodels
!pip install pydeflate
!pip install wbgapi
!pip install wbdata

DEPRECATION: pydeflate 1.0.1 has a non-standard dependency specifier pyarrow>=1.19.2xlrd>=2.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pydeflate or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pydeflate 1.0.1 has a non-standard dependency specifier pyarrow>=1.19.2xlrd>=2.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pydeflate or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pydeflate 1.0.1 has a non-standard dependency specifier pyarrow>=1.19.2xlrd>=2.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pydeflate or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pydeflate 1.0.1 has a non-standard dependency specifier pyarrow>=1.19.2xlrd>=2.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pydeflate or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pydeflate 1.0.1 has a non-standard dependency specifier pyarrow>=1.19.2xlrd>=2.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pydeflate or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pydeflate 1.0.1 has a non-standard dependency specifier pyarrow>=1.19.2xlrd>=2.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pydeflate or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pydeflate 1.0.1 has a non-standard dependency specifier pyarrow>=1.19.2xlrd>=2.0. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pydeflate or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


## **CHAPTER 1: DATA**

### OVERVIEW on DATA IMPORT & ADJUSTMENT REQUIREMENTS

#### DATA REQUIREMENTS
(N.B. the following table has not descriptive purposes, but it is rather intended as an auxiliary tool for the editors)

| Step | Imported? | Season adjusted? | Currency adjusted? | Variable | Variable Name | Shortcut | Note | Source | Link | Date |
|------|-----------|-------------------|---------------------|----------|---------------|----------|------|--------|------|------|
| 1    | Yes       | Yes               | Yes               | Yt       | Real GDP      | NGDPRSAXDCSAQ | Domestic Currency, SA, Quarterly | FRED | [Link](https://fred.stlouisfed.org/series/NGDPRSAXDCSAQ) | 2010 - current |
| 2    | Yes       |                   | Yes               | C        | Real Private Sector Final Consumption Expenditure | NCPRNSAXDCSAQ | Domestic Currency, Not SA, Quarterly | FRED | [Link](https://fred.stlouisfed.org/series/NCPRNSAXDCSAQ) | 2010 - current |
| 3    | Yes       | I tried           | USD                 | It       | Gross Fixed Capital Formation | NE.GDI.FTOT.CD | Current US Dollar, Annual | WorldBank | [Link](https://data.worldbank.org/indicator/NE.GDI.FTOT.CD?locations=SA) | 1968-2022 |
| 4    | Yes       | No                | USD                 | TBt      | Trade Balance | BN.GSR.GNFS.CD | Net trade in goods and services (BOP, current US$) | World Bank | [Link](https://data.worldbank.org/indicator/BN.GSR.GNFS.CD?locations=SA) | 1971-2022 |
| 5    | No        | Fuck this         | I am adjusted       | Nt       | Average Hours of Work for Employed Persons | Excel attached | Measures the average weekly working hours of workers (15 years and over) | General Statistics Authority | [Link](https://database.stats.gov.sa/home/indicator/999118) | Q2 2016-Q2 2023 |
| 6    | No        |                   |                     | wt       | Average Monthly Wages for Paid Employees | Excel attached | The sum of the monthly wages to the total of employed persons for wages | General Statistics Authority | [Link](https://database.stats.gov.sa/home/indicator/999118) | Q2 2016-Q2 2023 |
| 7    | Yes       |                   | In %                |          | Real Long Term Bond Rate | - | 10-year government development bond yield, Quarterly | National Statistics, Saudi Central Bank | [Link](https://www.sama.gov.sa/en-US/GovtSecurity/pages/governmentdevelopmentbonds.aspx), [Specific Bond (19Jan2029)](https://cbonds.com/bonds/461405/) | Q1999-Q32007, Past 3 yrs |
| 8    | Yes       |                   | In %                | rt       | Short Term Bond Rate | Excel attached | Quarterly average of interbank rate | Saudi Central Bank | [Link](https://www.sama.gov.sa/ar-sa/EconomicReports/Pages/report.aspx?cid=118) | 2007-current |
| 9    | Yes       |                   | Riyal               | Kt       | Capital Stock at Constant National Prices | RKNANPSAA666NRUG | Millions of 2017 U.S. Dollars, Not SA, Annual | FRED | [Link](https://fred.stlouisfed.org/series/RKNANPSAA666NRUG) | 1970-2019 |
| 10   | Yes       |                   | In %                | CAt      | Current Account | BN.CAB.XOKA.GD.ZS | Current account balance % of GDP, Annual | Worldbank | [Link](https://data.worldbank.org/indicator/BN.CAB.XOKA.GD.ZS?locations=SA) | 1970-2021 |
| 11   | Yes       |                   | In %                |          | CPI           | Consumer Prices | FP.CPI.TOTL.ZG | % Annual | Worldbank | [Link](https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?locations=SA) | 1964-2021 |
| 12   | Yes       |                   |                     | ER       | Exchange Rate  | - | The Saudi riyal has been at a fixed rate to the US dollar since June 1986 (SAR 3.7500 per USD) | Constant | - | - |
| 13   | Yes       | USD               |                     | Oil      | Crude Oil, Brent Prices | Excel attached | Crude oil, average spot price of Brent, Dubai and West Texas Intermediate, equally weighed, Monthly | Worldbank | [Link](https://www.worldbank.org/en/research/commodity-markets) | - |


### DATA MANIPULATION and CLEANING

#### 1) <u>Yt=Real Gross Domestic Product<u/>

##### 1.1) Reshaping & Seasonality correction

In [44]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pandas_datareader.fred import FredReader
data = FredReader(symbols=['NGDPRSAXDCSAQ'], start='2010', end=None).read()
data.to_csv('SA_gdp_seasonally_adjusted.csv')
# Read the CSV file and parse 'DATE' as datetime
df1 = pd.read_csv('SA_gdp_seasonally_adjusted.csv', parse_dates=['DATE'])
# Save for merging later 
df1.to_csv('SA_gdp_seasonally_adjusted.csv', index=False)
# Rename the columns
df1.columns = ['Date', 'Y']
# Display the DataFrame
df1.head()

,Date,Y
0,2010-01-01,472498.1
1,2010-04-01,487268.9
2,2010-07-01,492963.7
3,2010-10-01,521219.1
4,2011-01-01,535829.9


##### 1.2) Data format

In [41]:
# Check the format of the data so make sure we have a float variable
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    55 non-null     datetime64[ns]
 1   Y       55 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1008.0 bytes


##### 1.3) Index

In [42]:
# Create an index using our Date column
df1.set_index('Date', inplace=True)
df1.head()

,Y
Date,
2010-01-01,472498.1
2010-04-01,487268.9
2010-07-01,492963.7
2010-10-01,521219.1
2011-01-01,535829.9


##### 1.4) Log adjustment (stationary test)

In [45]:
# Run the Dickey-Fuller test  to see if we need to adjust our data using logs due to stationary/non-stationary
# Import necessary libraries
from pandas import read_csv
from statsmodels.tsa.stattools import adfuller
import numpy as np
from prettytable import PrettyTable
# Read the CSV file
series = read_csv('SA_gdp_seasonally_adjusted.csv', header=0, index_col=0)
# Original time series
X_original = series.values
# Log-transformed time series
X_log = np.log(df1.Y)
# Run the Dickey-Fuller test for the original time series
result_original = adfuller(X_original)
# Run the Dickey-Fuller test for the log-transformed time series
result_log = adfuller(X_log)
# Create a PrettyTable for the comparison
comparison_table = PrettyTable()
comparison_table.field_names = ["Statistic", "Original Series", "Log-Transformed Series"]
comparison_table.add_row(["ADF Statistic", result_original[0], result_log[0]])
comparison_table.add_row(["p-value", result_original[1], result_log[1]])
# Find the common keys for critical values
common_keys = set(result_original[4].keys()).intersection(result_log[4].keys())
# Add critical values to the table
for key in common_keys:
    comparison_table.add_row([f"Critical Value ({key})", result_original[4][key], result_log[4][key]])
# Print the comparison table
print(comparison_table)

+----------------------+---------------------+------------------------+
|      Statistic       |   Original Series   | Log-Transformed Series |
+----------------------+---------------------+------------------------+
|    ADF Statistic     |  -2.233491459545733 |   -2.791125939238542   |
|       p-value        |  0.1943152570602683 |  0.05955258781075299   |
| Critical Value (5%)  |  -2.925338105429433 |    -2.9178502070837    |
| Critical Value (1%)  | -3.5778480370438146 |   -3.560242358792829   |
| Critical Value (10%) | -2.6007735310095064 |  -2.5967964150943397   |
+----------------------+---------------------+------------------------+


##### The log-transformed time series, with a more negative ADF Statistic (-2.791) and a lower p-value (0.059) compared to the original series (-2.233 and 0.194), exhibits more compelling indications of stationarity. Given these Fuller test results, we opt for the log-transformed data for further analysis.

In [46]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    55 non-null     datetime64[ns]
 1   Y       55 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1008.0 bytes


#### 2) <u>C=Real Private Sector Final Consumption Expenditure<u/>

##### 2.1) Currency Conversion

In [ ]:
# Import necessary libraries
import pydeflate
import pandas as pd
# Add column to have identifier for SAUdi Dollars
df1['iso_code'] = 'SAU'
# Convert to USD 2023 constant prices
# Check if 'Y' and 'Date' are in the columns
if 'Y' in df1.columns:
    # If the columns exist, strip any leading or trailing whitespaces
    df1.columns = df1.columns.str.strip()
    # Reset the index to convert 'Date' from index to a regular column
    df1.reset_index(inplace=True)

    # do the conversion
    df1 = pydeflate.exchange(
        df=df1,
        source_currency="SAU",
        target_currency="USA",
        rates_source='imf', 
        value_column='Y',
        target_column='Y_USD',
        id_column="iso_code",
        id_type="ISO3",
        date_column="Date",
    )


df1.info()
df1.head

In [ ]:
### Step 2 ## Importing Real Private Sector Final Consumption Expenditure
## data is not seasonaly adjusted yet

data = FredReader(symbols=['NCPRNSAXDCSAQ'], start='2010', end=None).read()
data.to_csv('SA_real_private_consumption_not_adjusted.csv')

# Read the CSV file and parse 'DATE' as datetime
df2 = pd.read_csv('SA_real_private_consumption_not_adjusted.csv', parse_dates=['DATE'])

# save for merging later 
df2.to_csv('SA_real_private_consumption_not_adjusted.csv', index=False)

# Rename the columns
df2.columns = ['Date', 'C']

# Display the DataFrame
df2.head()


In [ ]:
# Check the format of data before merge
df2.info()
df2.plot()

In [ ]:
df2.C.map(np.log).diff().plot()

In [ ]:
df2['log_C'] = df2['C'].map(np.log)
df2['diff_log_C'] = df2['log_C'].diff()

# Include the 'Date' column from df2
d_log_C = df2[['Date']].copy()
d_log_C['diff_log_C'] = df2['diff_log_C']
d_log_C.info()


In [ ]:
# Extract quarter information
d_log_C['Quarter'] = d_log_C['Date'].dt.quarter

# Create a pivot table based on quarter and year
d_C_pivot = d_log_C.pivot_table(index='Quarter', columns=d_log_C['Date'].dt.year, values='diff_log_C')

# Display the pivot table
d_C_pivot.head()

In [ ]:
d_C_pivot.plot(legend=False);


Results seem not completly random but also not completly non random. Not adjusting for seasonality. 

In [ ]:
### not done with the correct C I assume as I renamed it to get log before. Has to be adjusted I believe
# add column to have identifier for SAudi Dollars
df2['iso_code'] = 'SAU'

# convert to USD 2023 constant prices
if 'C' in df2.columns:
    # If the column exists, strip any leading or trailing whitespaces
    df2.columns = df2.columns.str.strip()

    # Currency Conversion for C
    df2 = pydeflate.exchange(
        df=df2,
        source_currency="SAU",
        target_currency="USA",
        rates_source='imf', 
        value_column='C',  # Ensure the correct column name is used
        target_column='C_USD',
        id_column="iso_code",
        id_type="ISO3",
        date_column="Date",
    )

    # Currency Conversion for C_Log
    df2 = pydeflate.exchange(
        df=df2,
        source_currency="SAU",
        target_currency="USA",
        rates_source='imf', 
        value_column='log_C', 
        target_column='Log_C_USD',
        id_column="iso_code",
        id_type="ISO3",
        date_column="Date",
    )


df2.info()
df2.head

In [ ]:
!pip install wbdata
!pip install pandas_datareade

In [ ]:
### Step 3 ## Importing Gross Fixed Capital Formation NE.GDI.FTOT.CD Annual

## data is in current US Dollar

import pandas as pd
import pandas_datareader.wb as wb

# Define the World Bank indicator code for "Gross Fixed Capital Formation"
indicator_code = 'NE.GDI.FTOT.CD'

# Define countries and time period
countries = ['SA']
start_date = '2010-01-01'
end_date = '2023-12-31'

# Fetch World Bank data using the pandas_datareader library
world_bank_data = wb.download(indicator=indicator_code, country=countries, start=start_date, end=end_date)

# Reset index to make 'Date' a column
world_bank_data.reset_index(inplace=True)

# Drop 'country' column
world_bank_data.drop('country', axis=1, inplace=True)

# Rename 'year' column to 'Date'
world_bank_data.rename(columns={'year': 'Date', 'NE.GDI.FTOT.CD': 'I'}, inplace=True)
world_bank_data['Date'] = pd.to_datetime(world_bank_data['Date'])  # Fix: use 'year' instead of 'Date'

df3 = world_bank_data.copy()
df3.info()





In [ ]:
### resample data to fit quartelry framework
# Set 'Date' column as the index
df3.set_index('Date', inplace=True)

# Resample data to fit quarterly framework
df3_resampled = df3.I.resample('QS').asfreq()

# Plot the resampled data
df3_resampled.plot(marker='o')
df3.I.resample('QS').asfreq().plot(marker='o')

In [ ]:
# taking a loook at the NA
quarterly_I = df3.I.mul(0.25).resample('QS').asfreq()
quarterly_I.tail()

In [ ]:
import statsmodels.api as sm

# order is three numbers: (AR, Diff, MA). If this means nothing to you don't worry. But AR(1) corresponds to
# order (1, 0, 0).

# We set the trend to 'ctt' which means quadratic trend, since this data is increasing exponentially.
mod = sm.tsa.SARIMAX(quarterly_I, order=(1, 0, 0), trend='ctt')
res = mod.fit(method='nm', maxiter=10_000)

In [ ]:
res.summary()

In [ ]:
fig, ax = plt.subplots()
res.predict().plot(label='Predicted', ax=ax)
ax.scatter(quarterly_I.index, quarterly_I.values, s=15, color='tab:orange', label='Data')
ax.legend()
plt.show()

In [ ]:
# Plot the resampled data
df3.I.resample('QS').asfreq().plot(marker='o')

# Taking a look at the NA
quarterly_I = df3.I.mul(0.25).resample('QS').asfreq()

# Display the last few rows of the resampled quarterly data
print(quarterly_I.tail())

# SARIMAX Modeling
mod = sm.tsa.SARIMAX(quarterly_I, order=(1, 0, 0), trend='ctt')
res = mod.fit(method='nm', maxiter=10_000)

# Generate quarterly periods
quarterly_periods = pd.date_range(start=quarterly_I.index[-1] + pd.DateOffset(months=3), periods=4, freq='Q')

# Make predictions for quarterly data
predictions = res.get_prediction(start=quarterly_periods[0], end=quarterly_periods[-1])
predicted_data = predictions.predicted_mean

# Combine original quarterly data with predicted quarterly data
I_predict = pd.concat([quarterly_I, pd.DataFrame(predicted_data, columns=['I'], index=quarterly_periods)])

# Sort the combined data by the index (Date)
I_predict.sort_index(inplace=True)

# Reset index and rename to Date
I_predict.reset_index(inplace=True)
I_predict.rename(columns={'index': 'Date'}, inplace=True)

# Forward-fill NAs
I_predict.ffill(inplace=True)

print(I_predict)

In [ ]:
# Reset index and rename to match your requirements
#combined_data_I.reset_index(inplace=True)

# Drop the number in the column row and also drop 'I' column

I_predict.rename(columns={0: 'I_pred'}, inplace=True)
I_predict.drop(columns='I', axis=1, inplace=True)
I_predict.head()
I_predict.info()

In [ ]:
### Step 3.2 ## Change currency to Saudi Dollar petro money

In [ ]:
### Step 4 ## 4	TBt	Trade Balance trade in goods and services (BOP, current US$)

## data is in current US Dollar

import pandas as pd
import pandas_datareader.wb as wb

# Define World Bank indicator code for Trade Balance
indicator_code = 'BN.GSR.GNFS.CD'

# Define the countries and time period
countries = ['SA']
start_date = '2010-01-01'
end_date = '2023-12-31'

# Fetch World Bank data using pandas_datareader
world_bank_data = wb.download(indicator=indicator_code, country=countries, start=start_date, end=end_date)

# Reset index to build Date column
world_bank_data.reset_index(inplace=True)

# Drop country
world_bank_data.drop('country', axis=1, inplace=True)

# Rename year to Date
world_bank_data.rename(columns={'year': 'Date', 'BN.GSR.GNFS.CD': 'TB'}, inplace=True)
world_bank_data['Date'] = pd.to_datetime(world_bank_data['Date']) 

df4 = world_bank_data.copy()
df4.info()
df4.head()

In [ ]:
### Step 4.2 ## Trade Balance trade in goods and services (BOP, current US$)
# change to quarterly numbers

In [ ]:
### Step 5 ## Average Hours of Work for Employed Persons
#import pandas as pd

# this API import is killing me
sa_ave_hours_worked = 'https://database.stats.gov.sa/gastatapi/portal/api/v1/indicators/getData?format=@CSV&api=7563820c60aa782a9c85abcf47378b198fdbff06b72c12cdfe9ae32b623f2ac1ba97c1197512c692d0d8ef53d2ff3bd69fe848f7dd7c817472066e4a441f89496a67e3d726b1f710d3618b10d5e5360d41628cd6b78d97694aa88997f4f1a4abbf201543c0f6d2beec472c4957f9d4d8c9d0c7aa20a58f056496572745478d003b4e1f28b2ba7652e2883a8c120e8150cbdcbb3df0efb7b3c2ad003f07f60852'
# different  setup
df5 = pd.read_csv(sa_ave_hours_worked)
df5.to_csv('sa_ave_hours_worked.csv', index=False)
# rename year to date to stay consistent
df5.columns = ['EMPLYEES_OBSV', 'Date', 'TOTAL_HOURS_OBSV']
# not real date format but to stay consistent
df5['Date'] = pd.to_datetime(df5['Date'], format='%Y')
#adjust year_time to our quarterly date format
#df5['YEAR_TIME'] = pd.to_datetime(df5['YEAR_TIME'].astype(str) + '-1', format='%Y-Q%m')
#nevermind the format is not the biggest problem 

#import from files
#this is also not working. I dont Saudi Arabia
#ave_hour_down = r'C:\Users\mauri\Documents\GitHub\Uni-projects\ave_hour_down.csv'
#df5_down = pd.read_csv(ave_hour_down)
#df5_down.info()

df5.info()

In [ ]:
### Step 5 take a look at the data
# checking for missing values
#print(df5.isnull().sum())
# general overview
import matplotlib.pyplot as plt
import pandas as pd

# Set the display format for floating-point numbers
pd.set_option('display.float_format', '{:.2f}'.format)

# Summary statistics
summary_stats = df5.describe()

# min and max values
min_values = df5.min()
max_values = df5.max()

# histograms
df5.hist(figsize=(10, 8))
plt.suptitle('Histograms')

# Box plots
df5.boxplot(figsize=(10, 6))
plt.title('Box Plots')

# Scatter plots for each pair of variables
#numerical_columns = df5.select_dtypes(include='number').columns
#scatter_matrix = pd.plotting.scatter_matrix(df5[numerical_columns], figsize=(12, 12), marker='o', diagonal='hist')
#plt.suptitle('Scatter Plots', y=1.02)

# # Line plot for total hours worked per year
# plt.figure(figsize=(10, 6))
# plt.plot(df5['Date'], df5['TOTAL_HOURS_OBSV'], marker='o', linestyle='-', color='b')
# plt.title('Total Hours Worked Per Year')
# plt.xlabel('Year')
# plt.ylabel('Total Hours Worked')
# plt.grid(True)


# Sort the DataFrame by the 'YEAR_TIME' column
df5_sorted = df5.sort_values(by='Date')

# Line plot for total hours worked per year
plt.figure(figsize=(10, 6))
plt.plot(df5_sorted['Date'], df5_sorted['TOTAL_HOURS_OBSV'], marker='o', linestyle='-', color='b')
plt.title('Total Hours Worked Per Year')
plt.xlabel('Year')
plt.ylabel('Total Hours Worked')
plt.grid(True)
plt.show()

# Show plots
plt.show()

# Display the summary statistics, min, and max values
print("Summary Statistics:")
print(summary_stats)
print("\nMin Values:")
print(min_values)
print("\nMax Values:")
print(max_values)


In [ ]:
### Step 6 ## Average Monthly Wages for Paid Employees


# i still hate this thing
sa_ave_wage_empl = 'https://database.stats.gov.sa/gastatapi/portal/api/v1/indicators/getData?format=@CSV&api=7563820c60aa782a9c85abcf47378b19184536fdfc8572a64c787b576ad46259eeffe7eba57b098eed5e5e4b28fac9f836d7c0bbbbd42ea0c7b3cc324add0657119ea69136445d759e41292d3d6c6828c1bc253adb97eadf45df2a6d3e9286501ce082b9ca038d4bde5a30967da3fabe0004ff8c5f9a2081679b2a33d0fe9f662e714b02aeeff24da1fff252beec6f04bf31f754e930bee313b87c923777b6f8d7dad96afb83bdd2e2f5832caf804844'
df6 = pd.read_csv(sa_ave_wage_empl)
df6.to_csv('sa_ave_wage_empl.csv', index=False)
# rename year to date to stay consistent
df6.columns = ['Date','EMPLYEES_PAYED_OBSV', 'MONTHLY_WAGE_OBSV']
# not real date format but to stay consistent
df6['Date'] = pd.to_datetime(df5['Date'], format='%Y')
#adjust year_time to our quarterly date format
#df6['YEAR_TIME'] = pd.to_datetime(df5['YEAR_TIME'].astype(str) + '-1', format='%Y-Q%m')
#nevermind the format is not the biggest problem 


df6.info()
df6.head()

In [ ]:
### Step 6 ## Average Monthly Wages for Paid Employees
#change to quarterly numbers

In [ ]:
### Step 7 ## Real Long Term Bond Rate
## wrong years

df7.info()
df7.head()

In [ ]:
### Step 8 ## Short Term Bond Rate	
## quarterly numbers

df8.info()
df8.head()

In [ ]:
### Step 9 ## Capital Stock at Constant National Prices
## in USD
## quarterly numbers

data = FredReader(symbols=['NCPRNSAXDCSAQ'], start='2010', end=None).read()
data.to_csv('Cap_stock_at_Constant_National_Prices_USD.csv')

# Read the CSV file and parse 'DATE' as datetime
df9 = pd.read_csv('Cap_stock_at_Constant_National_Prices_USD.csv', parse_dates=['DATE'])

# save for merging later 
df9.to_csv('Cap_stock_at_Constant_National_Prices_USD.csv', index=False)

# Rename the columns
df9.columns = ['Date', 'K']

# Display the DataFrame
df9.head()
df9.info()


In [ ]:
### Step 10 ## Current Account	BN.CAB.XOKA.GD.ZS
#Current account balance % of GDP, Annual

## data is in percent

import pandas as pd
import pandas_datareader.wb as wb

# Define the World Bank indicator code for "Gross Fixed Capital Formation"
indicator_code = 'BN.CAB.XOKA.GD.ZS'

# Define the countries and time period you are interested in (only Saudi Arabia in this case)
countries = ['SA']
start_date = '2010-01-01'
end_date = '2023-12-31'

# Fetch World Bank data using the pandas_datareader library
world_bank_data = wb.download(indicator=indicator_code, country=countries, start=start_date, end=end_date)

# Reset index to make 'Date' a column
world_bank_data.reset_index(inplace=True)

# Drop 'country' column
world_bank_data.drop('country', axis=1, inplace=True)

# Rename 'year' column to 'Date'
world_bank_data.rename(columns={'year': 'Date', 'BN.CAB.XOKA.GD.ZS': 'CA'}, inplace=True)
world_bank_data['Date'] = pd.to_datetime(world_bank_data['Date'])  # Fix: use 'year' instead of 'Date'

df10 = world_bank_data.copy()
df10.info()
df10.head()

In [ ]:
### Step 10.2 ## Current Account	BN.CAB.XOKA.GD.ZS
#Current account balance % of GDP, Annual

## change to quarterly


In [ ]:
### Step 11 ## Consumer Prices	FP.CPI.TOTL.ZG
#Current account balance % of GDP, Annual

## data is in percent

import pandas as pd
import pandas_datareader.wb as wb

# Define the World Bank indicator code for "Gross Fixed Capital Formation"
indicator_code = 'FP.CPI.TOTL.ZG'

# Define the countries and time period you are interested in (only Saudi Arabia in this case)
countries = ['SA']
start_date = '2010-01-01'
end_date = '2023-12-31'

# Fetch World Bank data using the pandas_datareader library
world_bank_data = wb.download(indicator=indicator_code, country=countries, start=start_date, end=end_date)

# Reset index to make 'Date' a column
world_bank_data.reset_index(inplace=True)

# Drop 'country' column
world_bank_data.drop('country', axis=1, inplace=True)

# Rename 'year' column to 'Date'
world_bank_data.rename(columns={'year': 'Date', 'FP.CPI.TOTL.ZG': 'CPI'}, inplace=True)
world_bank_data['Date'] = pd.to_datetime(world_bank_data['Date'])  # Fix: use 'year' instead of 'Date'

df11 = world_bank_data.copy()
df11.info()
df11.head()

In [ ]:
### Step 11.2 ## Consumer Prices	FP.CPI.TOTL.ZG

## change to quarterly


In [ ]:
### Step 13 ## Oil prices  Prices

!pip install wbgapi 
import wbgapi as wb
wb.source.info()


In [ ]:
### Step 13 ## Oil prices  Prices

## in USD
## weekly numbers

data = FredReader(symbols=['DCOILWTICO'], start='2010', end=None).read()
data.to_csv('Cruide_oil_barrel_USD.csv')

# Read the CSV file and parse 'DATE' as datetime
df13 = pd.read_csv('Cruide_oil_barrel_USD.csv', parse_dates=['DATE'])

# save for merging later 
df13.to_csv('Cruide_oil_barrel_USD.csv', index=False)

# Rename the columns
df13.columns = ['Date', 'Oil']

# Display the DataFrame
df13.head()


In [ ]:
### step 13 # handle missing values
df13['Date'] = pd.to_datetime(df13['Date'])
df13.set_index('Date', inplace=True)
df13_filled = df13.fillna((df13.ffill() + df13.bfill()) / 2)

In [ ]:
### Step 13 ## Oil prices  Prices change to quarterly
df13_quarterly = df13_filled.resample('Q').mean()

## in USD
## weekly numbers

In [ ]:
df13_filled.Oil.plot(label='Weekly', legend=True)
df13_quarterly.Oil.plot(label='Quarterly', legend=True)

In [ ]:
wb.economy.metadata.get('SAU')
wb.series.info(q='

### Overview of Data that needs to get imported
| Import step | Variable | Variable Name | Shortcut | Note | Source | Link | Date |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | Yt | Real Gross Domestic Product | NGDPRSAXDCSAQ | Domestic Currency, SA, Quarterly | FRED | [Link](https://fred.stlouisfed.org/series/NGDPRSAXDCSAQ) | 2010 - current |
| 2 | C | Real Private Sector Final Consumption Expenditure | NCPRNSAXDCSAQ | Domestic Currency, Not SA, Quarterly | FRED | [Link](https://fred.stlouisfed.org/series/NCPRNSAXDCSAQ) | 2010 - current |
| 3 | It | Gross Fixed Capital Formation | NE.GDI.FTOT.CD | Current US Dollar, Annual | WorldBank | [Link](https://data.worldbank.org/indicator/NE.GDI.FTOT.CD?locations=SA) | 1968-2022 |
| 4 | TBt | Trade Balance | BN.GSR.GNFS.CD | Net trade in goods and services (BOP, current US$) | World Bank | [Link](https://data.worldbank.org/indicator/BN.GSR.GNFS.CD?locations=SA) | 1971-2022 |
| 5 | Nt | Average Hours of Work for Employed Persons | Excel attached | Measures the average weekly working hours of workers (15 years and over) | General Statistics Authority | [Link](https://database.stats.gov.sa/home/indicator/999118) | Q2 2016-Q2 2023 |
| 6 | wt | Average Monthly Wages for Paid Employees | Excel attached | The sum of the monthly wages to the total of employed persons for wages | General Statistics Authority | [Link](https://database.stats.gov.sa/home/indicator/999118) | Q2 2016-Q2 2023 |
| 7 | Rt | Real Long Term Bond Rate | | 10-year government development bond yield, Quarterly | National Statistics, Saudi Central Bank | [Link](https://www.sama.gov.sa/en-US/GovtSecurity/pages/governmentdevelopmentbonds.aspx), [Specific Bond (19Jan2029)](https://cbonds.com/bonds/461405/) | Q1999-Q32007, Past 3 yrs |
| 8 | rt | Short Term Bond Rate | Excel attached | Quarterly average of interbank rate | Saudi Central Bank | [Link](https://www.sama.gov.sa/ar-sa/EconomicReports/Pages/report.aspx?cid=118) | 2007-current |
| 9 | Kt | Capital Stock at Constant National Prices | RKNANPSAA666NRUG | Millions of 2017 U.S. Dollars, Not SA, Annual | FRED | [Link](https://fred.stlouisfed.org/series/RKNANPSAA666NRUG) | 1970-2019 |
| 10 | CAt | Current Account | BN.CAB.XOKA.GD.ZS | Current account balance % of GDP, Annual | Worldbank | [Link](https://data.worldbank.org/indicator/BN.CAB.XOKA.GD.ZS?locations=SA) | 1970-2018 |
| 11 | CPI | Consumer Prices | FP.CPI.TOTL.ZG | % Annual | Worldbank | [Link](https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?locations=SA) | 1964-2022 |
| 12 | ER | Exchange Rate | | The Saudi riyal has been at a fixed rate to the US dollar since June 1986 (SAR 3.7500 per USD) | Constant | - |
| 13 | Oil | Crude Oil, Brent Prices | Excel attached | Crude oil, average spot price of Brent, Dubai and West Texas Intermediate, equally weighed, Monthly | Worldbank | [Link](https://www.worldbank.org/en/research/commodity-markets) | 960-current |


In [ ]:
### last step merge all df frames
# Merge the first two DataFrames on the 'Date' column
df_merged = pd.merge(df1, df2, on='Date')

# Merge the result with the third DataFrame ('I_predict') on the 'Date' column
df_merged = pd.merge(df_merged, I_predict, on='Date')

In [ ]:
df_merged.head()
